<a href="https://colab.research.google.com/github/aanchalkhanna72/AML/blob/main/E22001_AML_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**E22001 Aanchal Khanna**

**E22011 Arav Sood**

**Assigment 1: Predicting the price of King County Houses using bagged regressor model and Random Forest Regressor Model.**

**Reading Data**

In [40]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor 
from sklearn.ensemble import RandomForestRegressor
import warnings 
warnings.filterwarnings("ignore")


In [2]:
#reading file - train data and test data separately
house_train=pd.read_csv("https://raw.githubusercontent.com/aanchalkhanna72/Datasets/main/kc_house_train_data.csv")
house_test=pd.read_csv("https://raw.githubusercontent.com/aanchalkhanna72/Datasets/main/kc_house_test_data.csv")
#house=pd.concat([house_train,house_test],axis=0) 

In [3]:
#reading column names and their data types
house_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9761 entries, 0 to 9760
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             9761 non-null   int64  
 1   date           9761 non-null   object 
 2   price          9761 non-null   float64
 3   bedrooms       9761 non-null   int64  
 4   bathrooms      9761 non-null   float64
 5   sqft_living    9761 non-null   int64  
 6   sqft_lot       9761 non-null   int64  
 7   floors         9761 non-null   float64
 8   waterfront     9761 non-null   int64  
 9   view           9761 non-null   int64  
 10  condition      9761 non-null   int64  
 11  grade          9761 non-null   int64  
 12  sqft_above     9761 non-null   int64  
 13  sqft_basement  9761 non-null   int64  
 14  yr_built       9761 non-null   int64  
 15  yr_renovated   9761 non-null   int64  
 16  zipcode        9761 non-null   int64  
 17  lat            9761 non-null   float64
 18  long    

**Feature Transformation**

In [4]:
#converting date object to datetime format 
house_train["Date"]=pd.to_datetime(house_train["date"])
house_test["Date"]=pd.to_datetime(house_test["date"])


In [5]:
#finding the year in which the most recent house was sold (Train)
house_train["Yr_sold"]=pd.DatetimeIndex(house_train["Date"]).year
np.max(house_train.Yr_sold)

2015

In [6]:
#finding the year in which the most recent house was sold (test)
house_test["Yr_sold"]=pd.DatetimeIndex(house_test["Date"]).year
np.max(house_test.Yr_sold)

2015

In [7]:
#transforming variables
#house age from yr_built for train data
house_train["house_age"]=2015-house_train["yr_built"] #assuming the data was collected in  2015 since the max dates the houses were sold were in 2015
#one hot encoding of zipcodes
dummies_train=pd.get_dummies(house_train["zipcode"])
house_train=house_train.join(dummies_train)


In [8]:
#house_test["zipcode"]=pd.get_dummies(house_test["zipcode"])
house_test["house_age"]=2015-house_test["yr_built"]
#one hot encoding of zipcodes
dummies_test=pd.get_dummies(house_test["zipcode"])
house_test=house_test.join(dummies_test)

In [9]:
#checking if the columns have been inserted correctly
house_train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9212900260,20140527T000000,468000.0,2,1.00,1160,6000,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,114101516,20140528T000000,310000.0,3,1.00,1430,19901,1.5,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6054650070,20141007T000000,400000.0,3,1.75,1370,9680,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#checking if the columns have been inserted correctly
house_test.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1175000570,20150312T000000,530000.0,5,2.00,1810,4850,1.5,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16000397,20141205T000000,189000.0,2,1.00,1200,9850,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,461000390,20140624T000000,687500.0,4,1.75,2330,5000,1.5,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7895500070,20150213T000000,240000.0,4,1.00,1220,8075,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
house_test.columns

Index([           'id',          'date',         'price',      'bedrooms',
           'bathrooms',   'sqft_living',      'sqft_lot',        'floors',
          'waterfront',          'view',     'condition',         'grade',
          'sqft_above', 'sqft_basement',      'yr_built',  'yr_renovated',
             'zipcode',           'lat',          'long', 'sqft_living15',
          'sqft_lot15',          'Date',       'Yr_sold',     'house_age',
                 98001,           98002,           98003,           98004,
                 98005,           98006,           98007,           98008,
                 98010,           98011,           98014,           98019,
                 98022,           98023,           98024,           98027,
                 98028,           98029,           98030,           98031,
                 98032,           98033,           98034,           98038,
                 98039,           98040,           98042,           98045,
                 98052,  

In [12]:
#dropping columns that are redundant or provide no information that is usable to the model
house_train=house_train.drop(["id","zipcode","lat","long","date","yr_built","Yr_sold","Date"],axis=1)

In [13]:
#dropping columns that are redundant or provide no information that is usable to the model
house_test=house_test.drop(["id","zipcode","lat","long","date","yr_built","Yr_sold","Date"],axis=1)

In [14]:
len(house_train.columns), len(house_test.columns) 

(86, 86)

In [15]:
house_train.columns

Index([        'price',      'bedrooms',     'bathrooms',   'sqft_living',
            'sqft_lot',        'floors',    'waterfront',          'view',
           'condition',         'grade',    'sqft_above', 'sqft_basement',
        'yr_renovated', 'sqft_living15',    'sqft_lot15',     'house_age',
                 98001,           98002,           98003,           98004,
                 98005,           98006,           98007,           98008,
                 98010,           98011,           98014,           98019,
                 98022,           98023,           98024,           98027,
                 98028,           98029,           98030,           98031,
                 98032,           98033,           98034,           98038,
                 98039,           98040,           98042,           98045,
                 98052,           98053,           98055,           98056,
                 98058,           98059,           98065,           98070,
                 98072,  

In [16]:
#defining target and predictors -- train set
y=house_train.price
X=house_train.drop(["price"],axis=1)
X.shape

(9761, 85)

In [17]:
#defining target and predictors -- test set
y_test = house_test.price
X_test = house_test.drop('price',axis=1)

In [20]:
X.shape,X_test.shape

((9761, 85), (2217, 85))

**Experimenting with Model Fitting**

**A. Fitting Bagging  Regressor**

In [29]:
#importing mape metric
from sklearn.metrics import mean_absolute_percentage_error

**Default Model:**

In [30]:
#fitting default bagged model
def_bag = BaggingRegressor()
def_bag.fit(X,y)
#predicting train and test values
def_pred_train=def_bag.predict(X)
def_pred_test=def_bag.predict(X_test)
#calculating train/test accuracy of default bagged model using mape
print("Training accuracy: ", 1-mean_absolute_percentage_error(y,def_pred_train),"Test accuracy: ", 1-mean_absolute_percentage_error(y_test,def_pred_test))

Training accuracy:  0.9275923710479314 Test accuracy:  0.8174174427156287


**Tuned Bagged Model:**

In [21]:
#using grid search method to select the best parameters
from  sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator = BaggingRegressor(), param_grid = {
    'n_estimators' : [20,80,100,150]
},n_jobs= -1, verbose = 1)
    

In [22]:
#fitting grid algo 
grid.fit(X,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=BaggingRegressor(), n_jobs=-1,
             param_grid={'n_estimators': [20, 80, 100, 150]}, verbose=1)

In [23]:
#printing the best parameter
grid.best_params_

{'n_estimators': 80}

In [24]:
#model fitting of tuned Bagging Regressor
bagged=BaggingRegressor(base_estimator=None, n_estimators=80, oob_score=True)
bagged.fit(X,y)

BaggingRegressor(n_estimators=80, oob_score=True)

In [31]:
#predicting train and test values
y_pred_train=bagged.predict(X)
y_pred = bagged.predict(X_test)


In [33]:
#calculating train/test accuracy using mape of tuned model
print("Training accuracy: ", 1-mean_absolute_percentage_error(y,y_pred_train),"Test accuracy: ", 1-mean_absolute_percentage_error(y_test,y_pred))

Training accuracy:  0.9360221692234594 Test accuracy:  0.822886402780908


In [27]:
#another metric for test score accuracy - OOB score (however, we already have test data here)
bagged.oob_score_

0.8163158914246349

**B. Random Forest Model**

**Default Model:**

In [35]:
#default random forest regressor
d_rf=RandomForestRegressor()
d_rf.fit(X,y)
#predict test values
y_pred_rf = d_rf.predict(X_test)
y_pred_train_rf=d_rf.predict(X)
#calculating train and test accuracy using mape
print("Training accuracy: ",1-mean_absolute_percentage_error(y,y_pred_train_rf),"Test accuracy: ",1-mean_absolute_percentage_error(y_test,y_pred_rf))


Training accuracy:  0.9363989376165507 Test accuracy:  0.823713428200728


**Tuned Model:**

In [ ]:
#creating a range of features to choose from
feature=np.arange(1,10)

In [ ]:
#grid search to select the best model
from sklearn.model_selection import GridSearchCV
gridrf=GridSearchCV(RandomForestRegressor(), cv=5, param_grid={"n_estimators":[100,150],"max_features":feature},n_jobs= -1, verbose = 1)

In [ ]:
#fitting the gridsearch model
gridrf.fit(X,y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_features': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'n_estimators': [100, 150]},
             verbose=1)

In [ ]:
#finding the best  parameters
gridrf.best_params_

{'max_features': 8, 'n_estimators': 100}

In [36]:
#fitting  the random forest model 
rf=RandomForestRegressor(n_estimators=150 ,max_features=8)
rf.fit(X,y)

RandomForestRegressor(max_features=8, n_estimators=150)

In [37]:
#finding train/test accuracy through mape
y_pred_rf = rf.predict(X_test)
y_pred_rf_train=rf.predict(X)
print("Training accuracy: ",1-mean_absolute_percentage_error(y,y_pred_rf_train),"Test accuracy: ",1-mean_absolute_percentage_error(y_test,y_pred_rf))


Training accuracy:  0.9328987283243113 Test accuracy:  0.8123399177263458


**Insights post-experimentation**

1. We observed that performing grid search and tuning the hyperparameters of bagged and random forest models did not give better results. In fact, there was hardly a 1 percentage point difference between the training/test accuracies of both default and tuned models. This shows that tuning does not seem to impact the train/test accuracies of the bagged and random forest models.

2. This experiment prompted us to again conduct an experiment with the base regressor and see its impact on the test/train accuracies.

**Tuning base estimators/regressors of both models:**

*Defining parameters of the base regressor:*

In [42]:
#range
depth=np.arange(1,15)

In [44]:
#performing gridsearch to tune the base regressor
grid=GridSearchCV(DecisionTreeRegressor(),param_grid={"max_depth":depth,"min_samples_leaf":[2,4,6,8],"min_samples_split":[2,4,6,8]},n_jobs=-1,cv=5)
grid.fit(X,y)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'min_samples_leaf': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 6, 8]})

In [45]:
#finding best parameters
grid.best_params_

{'max_depth': 12, 'min_samples_leaf': 2, 'min_samples_split': 6}

In [46]:
#defining  base estimator
base_dt=DecisionTreeRegressor(max_depth=12,min_samples_leaf=2,min_samples_split=6)

*A.Tuning Bagged Model with  new base estimator:*

In [47]:
#fitting bagged model
bagged=BaggingRegressor(base_estimator=base_dt)
bagged.fit(X,y)
#predicting train and test values
bagged_pred_train=bagged.predict(X)
bagged_pred_test=bagged.predict(X_test)
#train/test accuracy
print("Training accuracy: ",1-mean_absolute_percentage_error(y,bagged_pred_train),"Test accuracy: ",1-mean_absolute_percentage_error(y_test,bagged_pred_test))


Training accuracy:  0.816156934229439 Test accuracy:  0.7679951829138064


*B.Tuning Random Forest Model with  new base estimator:*

In [48]:
#fitting random forest model
rfr=RandomForestRegressor(max_depth=12,min_samples_leaf=2,min_samples_split=6)
rfr.fit(X,y)
#predicting train and test values
rfr_pred_train=rfr.predict(X)
rfr_pred_test=rfr.predict(X_test)
#train/test accuracy
print("Training accuracy: ",1-mean_absolute_percentage_error(y,rfr_pred_train),"Test accuracy: ",1-mean_absolute_percentage_error(y_test,rfr_pred_test))


Training accuracy:  0.8196407408367005 Test accuracy:  0.7711179190195139


As is visible with this experiment, tuning the base regressor in bagged model and changing hyperparameters of DTs in random forest model yet again yields similar test/train accuracies. Although the  training accuracy and difference between  train/test accuracies decreases, we see that the model learns less compared to the experiments before.

**Conclusion:**
Based on our experiments,  we can conclude that bagging models and random forest models yield similar results in terms of train/test accuracies. One model does not seem to outperform another. 